In [1]:
!pip install tritonclient[all] pysnooper requests Pillow attrdict

In [2]:
# 客户端请求
import datetime
import argparse
from functools import partial
import os
import sys

from PIL import Image
import numpy as np
from attrdict import AttrDict
import struct
import tritonclient.grpc.model_config_pb2 as mc
import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException
from tritonclient.utils import triton_to_np_dtype
import json,time
import pysnooper
# pip3 install Pillow
import requests
import base64


# @pysnooper.snoop()
def parse_model(model_metadata, model_config):
    """
    Check the configuration of a model to make sure it meets the
    requirements for an image classification network (as expected by
    this client)
    """
    if len(model_metadata.inputs) != 1:
        raise Exception("expecting 1 input, got {}".format(
            len(model_metadata.inputs)))
    if len(model_metadata.outputs) != 1:
        raise Exception("expecting 1 output, got {}".format(
            len(model_metadata.outputs)))

    if len(model_config.input) != 1:
        raise Exception(
            "expecting 1 input in model configuration, got {}".format(
                len(model_config.input)))

    input_metadata = model_metadata.inputs[0]
    input_config = model_config.input[0]
    output_metadata = model_metadata.outputs[0]

    if output_metadata.datatype != "FP32":
        raise Exception("expecting output datatype to be FP32, model '" +
                        model_metadata.name + "' output type is " +
                        output_metadata.datatype)

    # Output is expected to be a vector. But allow any number of
    # dimensions as long as all but 1 is size 1 (e.g. { 10 }, { 1, 10
    # }, { 10, 1, 1 } are all ok). Ignore the batch dimension if there
    # is one.
    output_batch_dim = (model_config.max_batch_size > 0)
    non_one_cnt = 0
    for dim in output_metadata.shape:
        if output_batch_dim:
            output_batch_dim = False
        elif dim > 1:
            non_one_cnt += 1
            if non_one_cnt > 1:
                raise Exception("expecting model output to be a vector")

    # Model input must have 3 dims, either CHW or HWC (not counting
    # the batch dimension), either CHW or HWC
    input_batch_dim = (model_config.max_batch_size > 0)
    expected_input_dims = 3 + (1 if input_batch_dim else 0)
    if len(input_metadata.shape) != expected_input_dims:
        raise Exception(
            "expecting input to have {} dimensions, model '{}' input has {}".
                format(expected_input_dims, model_metadata.name,
                       len(input_metadata.shape)))

    if type(input_config.format) == str:
        FORMAT_ENUM_TO_INT = dict(mc.ModelInput.Format.items())
        input_config.format = FORMAT_ENUM_TO_INT[input_config.format]

    if ((input_config.format != mc.ModelInput.FORMAT_NCHW) and
            (input_config.format != mc.ModelInput.FORMAT_NHWC)):
        raise Exception("unexpected input format " +
                        mc.ModelInput.Format.Name(input_config.format) +
                        ", expecting " +
                        mc.ModelInput.Format.Name(mc.ModelInput.FORMAT_NCHW) +
                        " or " +
                        mc.ModelInput.Format.Name(mc.ModelInput.FORMAT_NHWC))

    if input_config.format == mc.ModelInput.FORMAT_NHWC:
        h = input_metadata.shape[1 if input_batch_dim else 0]
        w = input_metadata.shape[2 if input_batch_dim else 1]
        c = input_metadata.shape[3 if input_batch_dim else 2]
    else:
        c = input_metadata.shape[1 if input_batch_dim else 0]
        h = input_metadata.shape[2 if input_batch_dim else 1]
        w = input_metadata.shape[3 if input_batch_dim else 2]

    return (model_config.max_batch_size, input_metadata.name,
            output_metadata.name, c, h, w, input_config.format,
            input_metadata.datatype)


def preprocess(img, format, dtype, c, h, w, scaling, protocol):
    """
    Pre-process an image to meet the size, type and format
    requirements specified by the parameters.
    """
    # np.set_printoptions(threshold='nan')

    if c == 1:
        sample_img = img.convert('L')
    else:
        sample_img = img.convert('RGB')

    resized_img = sample_img.resize((w, h), Image.BILINEAR)
    resized = np.array(resized_img)
    if resized.ndim == 2:
        resized = resized[:, :, np.newaxis]

    npdtype = triton_to_np_dtype(dtype)
    typed = resized.astype(npdtype)

    if scaling == 'INCEPTION':
        scaled = (typed / 127.5) - 1
    elif scaling == 'VGG':
        if c == 1:
            scaled = typed - np.asarray((128,), dtype=npdtype)
        else:
            scaled = typed - np.asarray((123, 117, 104), dtype=npdtype)
    else:
        scaled = typed

    # Swap to CHW if necessary
    if format == mc.ModelInput.FORMAT_NCHW:
        ordered = np.transpose(scaled, (2, 0, 1))
    else:
        ordered = scaled

    # Channels are in RGB order. Currently model configuration data
    # doesn't provide any information as to other channel orderings
    # (like BGR) so we just assume RGB.
    return ordered




In [22]:
model_name='resnet50'
model_version='202208014'
url = 'resnet50-202208014.service:8000'
classes=1000

triton_client = httpclient.InferenceServerClient(url=url, verbose=False, concurrency=1)

# 获取模型的配置信息
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)
print('模型信息：',model_metadata)


模型信息： {'name': 'resnet50', 'versions': ['202208014'], 'platform': 'onnxruntime_onnx', 'inputs': [{'name': 'input_name', 'datatype': 'FP32', 'shape': [3, 224, 224]}], 'outputs': [{'name': 'output_name', 'datatype': 'FP32', 'shape': [1000]}]}


In [23]:
model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)
print('模型配置：',model_config)


模型配置： {'name': 'resnet50', 'platform': 'onnxruntime_onnx', 'backend': 'onnxruntime', 'version_policy': {'latest': {'num_versions': 1}}, 'max_batch_size': 0, 'input': [{'name': 'input_name', 'data_type': 'TYPE_FP32', 'format': 'FORMAT_NCHW', 'dims': [3, 224, 224], 'reshape': {'shape': [1, 3, 224, 224]}, 'is_shape_tensor': False, 'allow_ragged_batch': False, 'optional': False}], 'output': [{'name': 'output_name', 'data_type': 'TYPE_FP32', 'dims': [1000], 'reshape': {'shape': [1, 1000]}, 'label_filename': '', 'is_shape_tensor': False}], 'batch_input': [], 'batch_output': [], 'optimization': {'priority': 'PRIORITY_DEFAULT', 'input_pinned_memory': {'enable': True}, 'output_pinned_memory': {'enable': True}, 'gather_kernel_buffer_threshold': 0, 'eager_batching': False}, 'instance_group': [{'name': 'resnet50', 'kind': 'KIND_CPU', 'count': 2, 'gpus': [], 'secondary_devices': [], 'profile': [], 'passive': False, 'host_policy': ''}], 'default_model_filename': 'model.onnx', 'cc_model_filenames': {

In [24]:
max_batch_size, input_name, output_name, c, h, w, format, dtype = parse_model(AttrDict(model_metadata), AttrDict(model_config))
# max_batch_size, input_name, output_name, c, h, w, format, dtype = 0,'input_name','output_name',3,224,224,2,'FP32'
print('请求参数：',max_batch_size, input_name, output_name, c, h, w, format, dtype)


请求参数： 0 input_name output_name 3 224 224 2 FP32


In [25]:
image_base64 ='...'
byte_content=base64.b64decode(image_base64)
filename = 'smallcat.jpg'
file = open(filename,mode='wb')
file.write(byte_content)
file.close()

![示例输入图片](smallcat.jpg)

In [26]:
image_data = preprocess(Image.open(filename), format, dtype, c, h, w, None,'http')
batched_image_data = image_data
print(batched_image_data)
# Send request
input = httpclient.InferInput(input_name, batched_image_data.shape, dtype)
input.set_data_from_numpy(batched_image_data)
output = httpclient.InferRequestedOutput(output_name, class_count=classes)

response = triton_client.infer(model_name,[input],request_id=str(time.time()),model_version=model_version,outputs=[output])
output_array = response.as_numpy(output_name)
print('响应：',len(output_array),output_array)

[[[255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  ...
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]]

 [[255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  ...
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]]

 [[255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  ...
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]]]
响应： 1000 [b'2146.753174:818' b'1593.821533:862' b'1580.508667:920'
 b'1497.157349:819' b'1446.738281:619' b'1390.397339:437'
 b'1384.842041:555' b'1364.949707:530' b'1294.385498:754'
 b'1257.208740:778' b'1255.856934:731' b'1251.588379:632'
 b'1234.268677:968' b'1149.677612:407' b'1127.52465